# Optuna

## Imports

In [1]:
import os
import pytorch_lightning as pl
import torch
import optuna
from optuna.integration import PyTorchLightningPruningCallback

# os.chdir(os.getcwd() + "/Code")
from src.Network import RegressionNet
os.chdir(os.path.dirname(os.getcwd()))

%load_ext autoreload
%autoreload 2

## Logger

In [2]:
class MetricsCallback(pl.Callback):
    """PyTorch Lightning metric callback."""
    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

## Objective

In [5]:
def objective(trial):
    # as explained above, we'll use this callback to collect the validation accuracies
    metrics_callback = MetricsCallback()  
    
    # create a trainer
    trainer = pl.Trainer(
        #train_percent_check=1.0,
        #val_percent_check=1.0,
        logger=False,                                                                  # deactivate PL logging
        max_epochs=32,                                                                  # epochs
        gpus=1 if torch.cuda.is_available() else None,                                 # #gpus
        callbacks=[metrics_callback],                                                  # save latest accuracy
        early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="val_loss"), # early stopping
    )

    # here we sample the hyper params, similar as in our old random search
    trial_hparams = {"days_of_data": trial.suggest_int("days_of_data", 14, 32),
                     "batch_size": 64, 
                     "lr": trial.suggest_loguniform("lr", 1e-6, 1e-1), 
                     "n_hidden": trial.suggest_int("n_hidden", 400, 1000), 
                     "optim": "SGD",
                     "loss": "MSE"
                            }
                        
    # create model from these hyper params and train it
    model = RegressionNet.RegressionNet(hparams=trial_hparams)
    trainer.fit(model)

    # save model
    # TODO
    # save_model(model, '{}.p'.format(trial.number), "checkpoints")

    # return validation accuracy from latest model, as that's what we want to minimize by our hyper param search
    return metrics_callback.metrics[-1]["val_loss"]

## Search

In [6]:
pruner = optuna.pruners.NopPruner()
study = optuna.create_study(direction="minimize", pruner=pruner)
study.optimize(objective, 
#                n_trials=512, 
#                timeout=600
              )

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


loss is MSE
Cuda available: True
Device: cuda:0
loading AAPL.csv
Dataset length: 9931
val_size: 1986
train_size: 7945
optim is SGD



  | Name    | Type       | Params
-----------------------------------
0 | model   | Sequential | 329 K 
1 | model.0 | Linear     | 65 K  
2 | model.1 | Tanh       | 0     
3 | model.2 | Linear     | 263 K 
4 | model.3 | Tanh       | 0     
5 | model.4 | Linear     | 514   
/home/thilo/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: Checkpoint directory /media/thilo/D68C8E958C8E7033/Users/threi/OneDrive/Thilo Reintjes/Projekte/Stock Datasets/checkpoints exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)


[I 2020-07-02 01:53:33,005] Finished trial#0 with value: 8.003308296203613 with parameters: {'days_of_data': 18, 'lr': 6.559087697016194e-05, 'n_hidden': 513}. Best is trial#0 with value: 8.003308296203613.
GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


loss is MSE
Cuda available: True
Device: cuda:0
loading AAPL.csv
Dataset length: 9917
val_size: 1983
train_size: 7934
optim is SGD


...

[W 2020-07-02 02:11:52,468] Setting status of trial#12 as TrialState.FAIL because the objective function returned nan.
GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


loss is MSE
Cuda available: True
Device: cuda:0
loading AAPL.csv
Dataset length: 9929
val_size: 1985
train_size: 7944
optim is SGD



  | Name    | Type       | Params
-----------------------------------
0 | model   | Sequential | 1 M   
1 | model.0 | Linear     | 135 K 
2 | model.1 | Tanh       | 0     
3 | model.2 | Linear     | 928 K 
4 | model.3 | Tanh       | 0     
5 | model.4 | Linear     | 964   


[W 2020-07-02 02:12:45,203] Setting status of trial#13 as TrialState.FAIL because of the following error: RuntimeError('DataLoader worker (pid 24058) is killed by signal: Aborted. ')
Traceback (most recent call last):
  File "/home/thilo/anaconda3/lib/python3.7/site-packages/optuna/study.py", line 734, in _run_trial
    result = func(trial)
  File "<ipython-input-5-421963ac54ac>", line 27, in objective
    trainer.fit(model)
  File "/home/thilo/anaconda3/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py", line 859, in fit
    self.single_gpu_train(model)
  File "/home/thilo/anaconda3/lib/python3.7/site-packages/pytorch_lightning/trainer/distrib_parts.py", line 503, in single_gpu_train
    self.run_pretrain_routine(model)
  File "/home/thilo/anaconda3/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py", line 1015, in run_pretrain_routine
    self.train()
  File "/home/thilo/anaconda3/lib/python3.7/site-packages/pytorch_lightning/trainer/training_loop.py", li

RuntimeError: DataLoader worker (pid 24058) is killed by signal: Aborted. 

## Check the Results

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
best_trial = study.best_trial

print("  Value: {}".format(best_trial.value))

print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
best_model = load_model('checkpoints/{}.p'.format(best_trial.number))
best_model.prepare_data()
test_and_save(best_model)

In [ ]:
shutil.rmtree("checkpoints")